In [ ]:
import pandas as pd
import glob

In [ ]:
# Get a list of file paths for all CSV files in the data/ folder
csv_files = glob.glob('../data/*.csv')

# Create an empty list to store the DataFrames
dfs = []

# Read each CSV file into a DataFrame and append it to the list
for file in csv_files:
    df = pd.read_csv(file)
    df["user"] = file.split("/")[-1].split(".")[0] # Extract the user name from the file path
    dfs.append(df)

# Concatenate all the DataFrames in the list
df = pd.concat(dfs)

print(f"Data points: {len(df)}")
df.head()

In [ ]:
display(df.describe())
df.hist(bins=100, figsize=(20, 15))

In [ ]:
num_past_reviews = 4

df["delta_t_0"] = df["delta_t"]
df["rating_0"] = df["rating"]
df = df[
    [
        "user",
        "card_id",
        "rating_0",
        "delta_t_0",
    ]
]

# Add columns for the past reviews
for i in range(1, num_past_reviews):
    df[f"delta_t_{i}"] = (
        df["delta_t_0"]
            .shift(i)
            .where(
                (df["user"].shift(i) == df["user"]) & (df["card_id"].shift(i) == df["card_id"])
            )
    )
    df[f"rating_{i}"] = (
        df["rating_0"]
            .shift(i)
            .where(
                (df["user"].shift(i) == df["user"]) & (df["card_id"].shift(i) == df["card_id"])
            )
    )

display(df.head(100).style.format())